In [19]:
file_date = '2021-04-18'

<div style="max-width:1400px;margin-center: auto">
<img src="images\race join.png" width="600"/>
</div>

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local') \
    .appName("race_join") \
    .config("spark.sql.catalogImplementation", "hive") \
    .getOrCreate()

In [21]:
races_df = spark.read.csv(f'raw files\\{file_date}\\races.csv', header =True, inferSchema =True).\
            withColumnRenamed('year','race_year').\
            withColumnRenamed('name','race_name').\
            withColumnRenamed('date','race_date')
constructors_df = spark.read.parquet(f'f1_processed\\constructors').\
            withColumnRenamed('name','team')
drivers_df = spark.read.parquet(f'f1_processed\\drivers').\
            withColumnRenamed('name','driver_name').\
            withColumnRenamed('number','driver_number').\
            withColumnRenamed('nationality','driver_nationality')
circuit_df = spark.read.parquet(f'f1_processed\\circuits').\
                    withColumnRenamed('location','circuit_location')
results_df = spark.read.parquet(f'f1_processed\\results') \
            .filter(f"file_date = '{file_date}'") \
            .withColumnRenamed("time", "race_time") \
            .withColumnRenamed("race_id", "result_race_id") \
            .withColumnRenamed("file_date", "result_file_date") 
#filter used to process only the data for the current date
races_df.show(2)
constructors_df.show(2)
drivers_df.show(2)
circuit_df.show(2)
results_df.show(2)

In [22]:
from pyspark.sql.functions import current_timestamp

In [23]:
fnl_df = results_df.join(constructors_df, results_df.constructor_id == constructors_df.constructor_id,'inner').\
        join(drivers_df,results_df.driver_id == drivers_df.driver_id,'inner').\
        join(races_df,results_df.result_race_id == races_df.raceId,'inner').\
        join(circuit_df,races_df.circuitId == circuit_df.circuit_id,'inner').\
        select(races_df.race_year,
               races_df.raceId,
               races_df.race_name,
               races_df.race_date,
               circuit_df.circuit_location,
               drivers_df.driver_name,
               drivers_df.driver_number,
               drivers_df.driver_nationality,
               constructors_df.team,
               results_df.grid,
               results_df.fastest_lap,
               results_df.race_time,
               results_df.points,
              results_df.position).withColumn("created_date", current_timestamp()) \
                          .withColumnRenamed("result_file_date", "file_date")\
                          .withColumnRenamed("raceId", "race_id")
fnl_df.show(2)

In [24]:
%run "common_functions.ipynb"

In [25]:
overwrite_partition(fnl_df, 'f1_trans', 'race_results', 'race_id')

In [6]:
df= spark.sql('SELECT race_id,count(*) FROM f1_trans.race_results GROUP BY race_id ORDER BY race_id DESC')
df.show()

+-------+--------+
|race_id|count(1)|
+-------+--------+
|   1053|      20|
|   1052|      20|
|   1047|      20|
|   1046|      20|
|   1045|      20|
|   1044|      20|
|   1043|      20|
|   1042|      20|
|   1041|      20|
|   1040|      20|
|   1039|      20|
|   1038|      20|
|   1037|      20|
|   1036|      20|
|   1035|      20|
|   1034|      20|
|   1033|      20|
|   1032|      20|
|   1031|      20|
|   1030|      20|
+-------+--------+
only showing top 20 rows

